In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
url = 'https://movie.douban.com/tag/#/?sort=S&range=2,10&tags=%E7%94%B5%E8%A7%86%E5%89%A7&start=0&countries=%E4%B8%AD%E5%9B%BD%E5%A4%A7%E9%99%86'
ips = ['122.152.196.126', '114.215.174.227', '119.185.30.75']
User_Agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
    'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0',
    'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
    'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11'
]
Headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 
    'Accept-Encoding': 'gzip, deflate, br', 
    'Accept-Language': 'zh-CN,zh;q=0.9', 
    'Connection': 'keep-alive', 
    'Cookie': 'll="108090"; bid=RCQuJh5W6uA; _vwo_uuid_v2=DE0655BABE1957865C0B256950AD3853D|0a7c48214e00a9dfb4577f1ac49ca58c; ps=y; push_noty_num=0; push_doumail_num=0; __utmv=30149280.18727; __utmc=30149280; gr_user_id=f19fd5f6-ecf0-49d8-a885-034037293bb9; as="https://sec.douban.com/b?r=https%3A%2F%2Fmovie.douban.com%2Fcelebrity%2F1312700%2F"; dbcl2="187278528:y9OXvoexX0g"; ck=xb-g; __yadk_uid=qVMgftxxu2SWyXjlxh6A6NYoDRYwQNnd; ap_v=0,6.0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1542950923%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3DVbj4lDLMZY4h9YwHoS4hPewratj0oD9JyXqsH-7oEmGNMOh90p3FMxx4p2--pOfb%26wd%3D%26eqid%3D9b06374a0000964e000000065bf78ff5%22%5D; _pk_ses.100001.8cb4=*; __utma=30149280.659914314.1541418835.1542948737.1542950925.24; __utmz=30149280.1542950925.24.5.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmt=1; douban-profile-remind=1; _pk_id.100001.8cb4=9aa7e0a808761a64.1541418995.6.1542951038.1542948737.; __utmb=30149280.6.10.1542950925', 
    'Host': 'www.douban.com', 
    'Referer': 'https://www.douban.com/people/187278528/', 
    'Upgrade-Insecure-Requests': '1', 
    'User-Agent': random.choice(User_Agents)
}

In [3]:
# driver = webdriver.Chrome()
# driver.get(url)
# driver.maximize_window()

In [4]:
def one_click():
    driver.find_element_by_class_name('more').click()
    time.sleep(1.5)
while True:
    try:
        one_click()
    except:
        break

In [5]:
# driver.find_element_by_class_name('th-list').click()

In [6]:
# urls = [k.get_attribute('href') for k in driver.find_elements_by_class_name('title')]

In [7]:
df = pd.read_csv('dataset/data_collection/tw_urls.csv')
urls = [url[0] for url in df.values]

In [8]:
def scrape_one_url(url):
    r = requests.get(url, proxies = {'http': random.choice(ips)}, headers = Headers)
    data = BeautifulSoup(r.text, 'html.parser')
    my1 = data.find('h1').text.strip()
    my2 = data.find(id = 'info').text.strip()
    my3 = data.find(typeof = 'v:Rating').text.strip()
    return my1, my2, my3
datasets = []
error_urls = []
# for url in urls:
#     try:
#         datasets.append(scrape_one_url(url))
#     except:
#         error_urls.append(url)

<font face="黑体" size=4 color=#3A5FCD > Scrape all Chinese TV dramas' link to their detail information page.

In [9]:
titles = []
years = []
for dataset in datasets:
    titles.append(dataset[0].split()[0])
    years.append(dataset[0].split()[-1].strip('()'))
df = pd.DataFrame({'剧名': titles, '年份': years})

In [10]:
headers = ['导演', '编剧', '主演', '类型', '制片国家/地区', '集数', '单集片长']
def add_one_df(string):
    things = []
    for dataset in datasets:
        thing = None
        for i in dataset[1].split('\n'):
            if i.find(string) != -1:
                thing = i.strip(string).strip(': ').strip('分钟')
                break
        things.append(thing)
    return things
for header in headers:
    df[header] = add_one_df(header)

In [11]:
scores = []
numbers = []
for dataset in datasets:
    scores.append(dataset[2].split('\n')[0])
    numbers.append(dataset[2].split('\n')[-1].strip('人评价'))
df['豆瓣评分'] = scores
df['评价人数'] = numbers

In [12]:
df.columns = ['剧名', '年份', '导演', '编剧', '主演', '类型', '制片国家/地区', '集数', '单集片长/分钟', '豆瓣评分', '评价人数/人']

In [13]:
final = df.sort_values(by = '豆瓣评分', ascending = False)

In [14]:
# final.to_csv('dataset/data_presentation/results_tw.csv', index = False, encoding = 'utf-8-sig')

<font face="黑体" size=4 color=#3A5FCD > Scrape basic information of all Chinese TV dramas.